In [1]:
import torch
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import numpy as np
import torch.nn as nn
from math import *

In [2]:
torch.cuda.set_device(1)

In [3]:
torch.set_default_tensor_type('torch.DoubleTensor') # 设置浮点类型为 torch.float64

In [4]:
# 定义激活函数: swish(x)
def acti(x):
    return x*torch.sigmoid(x)  

In [5]:
# 定义网络结构
class DeepRitzNet(torch.nn.Module):
    def __init__(self, input_width, layer_width):
        super(DeepRitzNet, self).__init__()
        self.linear_in = torch.nn.Linear(input_width, layer_width)
        self.linear1 = torch.nn.Linear(layer_width, layer_width)
        self.linear2 = torch.nn.Linear(layer_width, layer_width)
        self.linear3 = torch.nn.Linear(layer_width, layer_width)
        self.linear4 = torch.nn.Linear(layer_width, layer_width)
        self.linear5 = torch.nn.Linear(layer_width, layer_width)
        self.linear6 = torch.nn.Linear(layer_width, layer_width)
        self.linear_out = torch.nn.Linear(layer_width, 1)

    def forward(self, x):
        y = self.linear_in(x) # fully connect layer
        y = y + acti(self.linear2(acti(self.linear1(y)))) # residual block 1
        y = y + acti(self.linear4(acti(self.linear3(y)))) # residual block 2
        y = y + acti(self.linear6(acti(self.linear5(y)))) # residual block 3
        output = self.linear_out(y) # fully connect layer
        return output

In [6]:
dimension = 4

In [7]:
# exact solution
def u_ex(x):  
    x_temp = torch.cos(pi*x)
    u_x = (x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return u_x

In [8]:
def f(x):
    x_temp = torch.cos(pi*x)
    f_x = 2*pi**2*(x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return f_x

In [9]:
def DRM(x):
    u_hat = model(x)
    ux = torch.zeros(x.size()[0], dimension).cuda()
    step_size = 0.0001
    for i in range(dimension):
        dx = torch.zeros(x.size()[0], dimension).cuda()
        dx[:, i] = torch.ones(x.size()[0])
        ux[:, i] = (model(x+step_size*dx) - model(x-step_size*dx))[:,0]/step_size/2
    uxsq = (torch.sum(ux**2, dim = 1)).reshape([x.size()[0], 1]) # dim = 1 按行求和
    f_temp = f(x)
    part_1 = torch.sum(0.5 * uxsq + 0.5*pi**2*u_hat**2 - f_temp*u_hat)/x.size()[0]
    
    Nb = 100
    xa1 = torch.rand(Nb, dimension).cuda() 
    xa1[:, 0] = torch.zeros(Nb)
    xa2 = torch.rand(Nb, dimension).cuda() 
    xa2[:, 0] = torch.ones(Nb)
    xb1 = torch.rand(Nb, dimension).cuda()
    xb1[:, 1] = torch.zeros(Nb)
    xb2 = torch.rand(Nb, dimension).cuda()
    xb2[:, 1] = torch.ones(Nb)
    xc1 = torch.rand(Nb, dimension).cuda()
    xc1[:, 2] = torch.zeros(Nb)
    xc2 = torch.rand(Nb, dimension).cuda()
    xc2[:, 2] = torch.ones(Nb)
    xd1 = torch.rand(Nb, dimension).cuda()
    xd1[:, 3] = torch.zeros(Nb)
    xd2 = torch.rand(Nb, dimension).cuda()
    xd2[:, 3] = torch.ones(Nb)
    sa1 = (torch.sum((model(xa1)-u_ex(xa1))**2)/xa1.size()[0]).cuda()
    sa2 = (torch.sum((model(xa2)-u_ex(xa2))**2)/xa1.size()[0]).cuda() 
    sb1 = (torch.sum((model(xb1)-u_ex(xb1))**2)/xa1.size()[0]).cuda() 
    sb2 = (torch.sum((model(xb2)-u_ex(xb2))**2)/xa1.size()[0]).cuda()
    sc1 = (torch.sum((model(xc1)-u_ex(xc1))**2)/xa1.size()[0]).cuda() 
    sc2 = (torch.sum((model(xc2)-u_ex(xc2))**2)/xa1.size()[0]).cuda()
    sd1 = (torch.sum((model(xd1)-u_ex(xd1))**2)/xa1.size()[0]).cuda() 
    sd2 = (torch.sum((model(xd2)-u_ex(xd2))**2)/xa1.size()[0]).cuda()
    part_2 = sa1 + sa2 + sb1 + sb2 + sc1 + sc2 + sd1 + sd2
    
    lambda1 = 0.1
    return part_1 + lambda1 * part_2 / 8

In [10]:
Data_size = 2000
def Gendata():
    x = torch.rand(Data_size, dimension)
    return x.cuda()

In [11]:
# 正态分布初始化参数
def initparam(model, sigma):
    for m in model.modules():
        if isinstance(m, nn.Linear):
            m.weight.data.normal_(0, sigma)
    return model

In [12]:
model = DeepRitzNet(dimension, 8)
model = initparam(model, 0.5)

In [13]:
device = torch.device("cuda:1" )
model.to(device)

DeepRitzNet(
  (linear_in): Linear(in_features=4, out_features=8, bias=True)
  (linear1): Linear(in_features=8, out_features=8, bias=True)
  (linear2): Linear(in_features=8, out_features=8, bias=True)
  (linear3): Linear(in_features=8, out_features=8, bias=True)
  (linear4): Linear(in_features=8, out_features=8, bias=True)
  (linear5): Linear(in_features=8, out_features=8, bias=True)
  (linear6): Linear(in_features=8, out_features=8, bias=True)
  (linear_out): Linear(in_features=8, out_features=1, bias=True)
)

In [14]:
import torch.optim as optim
import torch.nn as nn
import time

In [15]:
def relative_error():
    x = Gendata()
    predict = model(x)
    exact = u_ex(x)
    value = torch.sqrt(torch.sum((predict - exact )**2))/torch.sqrt(torch.sum((exact )**2))
    return value

In [16]:
traintime = 20000
error_save = np.zeros(traintime)
optimizer = optim.Adam(model.parameters())

In [17]:
time_start = time.time()
for i in range(traintime):
    optimizer.zero_grad()
    x = Gendata()
    x.requires_grad = True
    losses = DRM(x)
    losses.backward()
    optimizer.step()
    error = relative_error()
    error_save[i] = float(error)
    
    if i % 50 == 0:
        print("current epoch is: ", i)
        print("current loss is: ", losses.detach())
        print("current relative error is: ", error.detach())
        np.save("DRM_relative_error_4D_Dirichlet_lambda0.1.npy", error_save)
np.save("DRM_relative_error_4D_Dirichlet_lambda0.1.npy", error_save)
time_end = time.time()
print('total time is: ', time_end-time_start, 'seconds')

current epoch is:  0
current loss is:  tensor(4.0130, device='cuda:1')
current relative error is:  tensor(1.2996, device='cuda:1')
current epoch is:  50
current loss is:  tensor(-10.0229, device='cuda:1')
current relative error is:  tensor(0.7409, device='cuda:1')
current epoch is:  100
current loss is:  tensor(-17.8553, device='cuda:1')
current relative error is:  tensor(0.2371, device='cuda:1')
current epoch is:  150
current loss is:  tensor(-18.7108, device='cuda:1')
current relative error is:  tensor(0.1681, device='cuda:1')
current epoch is:  200
current loss is:  tensor(-16.6603, device='cuda:1')
current relative error is:  tensor(0.1477, device='cuda:1')
current epoch is:  250
current loss is:  tensor(-18.3080, device='cuda:1')
current relative error is:  tensor(0.1435, device='cuda:1')
current epoch is:  300
current loss is:  tensor(-18.1024, device='cuda:1')
current relative error is:  tensor(0.1307, device='cuda:1')
current epoch is:  350
current loss is:  tensor(-17.3742, de

current epoch is:  3050
current loss is:  tensor(-19.4915, device='cuda:1')
current relative error is:  tensor(0.0588, device='cuda:1')
current epoch is:  3100
current loss is:  tensor(-19.8188, device='cuda:1')
current relative error is:  tensor(0.0194, device='cuda:1')
current epoch is:  3150
current loss is:  tensor(-19.4039, device='cuda:1')
current relative error is:  tensor(0.0281, device='cuda:1')
current epoch is:  3200
current loss is:  tensor(-18.7019, device='cuda:1')
current relative error is:  tensor(0.0990, device='cuda:1')
current epoch is:  3250
current loss is:  tensor(-19.2757, device='cuda:1')
current relative error is:  tensor(0.0251, device='cuda:1')
current epoch is:  3300
current loss is:  tensor(-20.4593, device='cuda:1')
current relative error is:  tensor(0.0257, device='cuda:1')
current epoch is:  3350
current loss is:  tensor(-18.7578, device='cuda:1')
current relative error is:  tensor(0.0318, device='cuda:1')
current epoch is:  3400
current loss is:  tensor

current epoch is:  6100
current loss is:  tensor(-19.2571, device='cuda:1')
current relative error is:  tensor(0.0227, device='cuda:1')
current epoch is:  6150
current loss is:  tensor(-20.6890, device='cuda:1')
current relative error is:  tensor(0.0548, device='cuda:1')
current epoch is:  6200
current loss is:  tensor(-19.4124, device='cuda:1')
current relative error is:  tensor(0.0352, device='cuda:1')
current epoch is:  6250
current loss is:  tensor(-19.0102, device='cuda:1')
current relative error is:  tensor(0.0362, device='cuda:1')
current epoch is:  6300
current loss is:  tensor(-19.8542, device='cuda:1')
current relative error is:  tensor(0.0390, device='cuda:1')
current epoch is:  6350
current loss is:  tensor(-19.6760, device='cuda:1')
current relative error is:  tensor(0.0195, device='cuda:1')
current epoch is:  6400
current loss is:  tensor(-18.9407, device='cuda:1')
current relative error is:  tensor(0.0558, device='cuda:1')
current epoch is:  6450
current loss is:  tensor

current epoch is:  9150
current loss is:  tensor(-19.6938, device='cuda:1')
current relative error is:  tensor(0.0333, device='cuda:1')
current epoch is:  9200
current loss is:  tensor(-19.0459, device='cuda:1')
current relative error is:  tensor(0.0343, device='cuda:1')
current epoch is:  9250
current loss is:  tensor(-19.0195, device='cuda:1')
current relative error is:  tensor(0.0193, device='cuda:1')
current epoch is:  9300
current loss is:  tensor(-18.8573, device='cuda:1')
current relative error is:  tensor(0.0277, device='cuda:1')
current epoch is:  9350
current loss is:  tensor(-19.8306, device='cuda:1')
current relative error is:  tensor(0.0344, device='cuda:1')
current epoch is:  9400
current loss is:  tensor(-20.3712, device='cuda:1')
current relative error is:  tensor(0.0242, device='cuda:1')
current epoch is:  9450
current loss is:  tensor(-19.1541, device='cuda:1')
current relative error is:  tensor(0.0255, device='cuda:1')
current epoch is:  9500
current loss is:  tensor

current epoch is:  12150
current loss is:  tensor(-19.5432, device='cuda:1')
current relative error is:  tensor(0.0300, device='cuda:1')
current epoch is:  12200
current loss is:  tensor(-18.2024, device='cuda:1')
current relative error is:  tensor(0.0219, device='cuda:1')
current epoch is:  12250
current loss is:  tensor(-18.5985, device='cuda:1')
current relative error is:  tensor(0.0226, device='cuda:1')
current epoch is:  12300
current loss is:  tensor(-19.3974, device='cuda:1')
current relative error is:  tensor(0.0223, device='cuda:1')
current epoch is:  12350
current loss is:  tensor(-19.7903, device='cuda:1')
current relative error is:  tensor(0.0220, device='cuda:1')
current epoch is:  12400
current loss is:  tensor(-21.7137, device='cuda:1')
current relative error is:  tensor(0.0410, device='cuda:1')
current epoch is:  12450
current loss is:  tensor(-20.6193, device='cuda:1')
current relative error is:  tensor(0.0273, device='cuda:1')
current epoch is:  12500
current loss is:

current epoch is:  15150
current loss is:  tensor(-21.5087, device='cuda:1')
current relative error is:  tensor(0.0319, device='cuda:1')
current epoch is:  15200
current loss is:  tensor(-19.2534, device='cuda:1')
current relative error is:  tensor(0.0233, device='cuda:1')
current epoch is:  15250
current loss is:  tensor(-18.7531, device='cuda:1')
current relative error is:  tensor(0.0394, device='cuda:1')
current epoch is:  15300
current loss is:  tensor(-20.3279, device='cuda:1')
current relative error is:  tensor(0.0470, device='cuda:1')
current epoch is:  15350
current loss is:  tensor(-19.4110, device='cuda:1')
current relative error is:  tensor(0.0167, device='cuda:1')
current epoch is:  15400
current loss is:  tensor(-21.3566, device='cuda:1')
current relative error is:  tensor(0.0305, device='cuda:1')
current epoch is:  15450
current loss is:  tensor(-19.8885, device='cuda:1')
current relative error is:  tensor(0.0664, device='cuda:1')
current epoch is:  15500
current loss is:

current epoch is:  18150
current loss is:  tensor(-20.0999, device='cuda:1')
current relative error is:  tensor(0.0387, device='cuda:1')
current epoch is:  18200
current loss is:  tensor(-21.0699, device='cuda:1')
current relative error is:  tensor(0.0446, device='cuda:1')
current epoch is:  18250
current loss is:  tensor(-19.9244, device='cuda:1')
current relative error is:  tensor(0.0513, device='cuda:1')
current epoch is:  18300
current loss is:  tensor(-19.9117, device='cuda:1')
current relative error is:  tensor(0.0850, device='cuda:1')
current epoch is:  18350
current loss is:  tensor(-19.1759, device='cuda:1')
current relative error is:  tensor(0.0271, device='cuda:1')
current epoch is:  18400
current loss is:  tensor(-19.1821, device='cuda:1')
current relative error is:  tensor(0.0274, device='cuda:1')
current epoch is:  18450
current loss is:  tensor(-19.6227, device='cuda:1')
current relative error is:  tensor(0.0391, device='cuda:1')
current epoch is:  18500
current loss is:

In [18]:
error_save_append = np.zeros(traintime)
time_start = time.time()
for i in range(traintime):
    optimizer.zero_grad()
    x = Gendata()
    x.requires_grad = True
    losses = DRM(x)
    losses.backward()
    optimizer.step()
    error = relative_error()
    error_save_append[i] = float(error)
    
    if i % 50 == 0:
        print("current epoch is: ", i)
        print("current loss is: ", losses.detach())
        print("current relative error is: ", error.detach())
        np.save("DRM_relative_error_4D_Dirichlet_lambda0.1_append.npy", error_save_append)
np.save("DRM_relative_error_4D_Dirichlet_lambda0.1_append.npy", error_save_append)
time_end = time.time()
print('total time is: ', time_end-time_start, 'seconds')

current epoch is:  0
current loss is:  tensor(-20.2993, device='cuda:1')
current relative error is:  tensor(0.0253, device='cuda:1')
current epoch is:  50
current loss is:  tensor(-19.7438, device='cuda:1')
current relative error is:  tensor(0.0331, device='cuda:1')
current epoch is:  100
current loss is:  tensor(-18.6960, device='cuda:1')
current relative error is:  tensor(0.0181, device='cuda:1')
current epoch is:  150
current loss is:  tensor(-19.5099, device='cuda:1')
current relative error is:  tensor(0.0161, device='cuda:1')
current epoch is:  200
current loss is:  tensor(-18.5261, device='cuda:1')
current relative error is:  tensor(0.0270, device='cuda:1')
current epoch is:  250
current loss is:  tensor(-19.3368, device='cuda:1')
current relative error is:  tensor(0.0255, device='cuda:1')
current epoch is:  300
current loss is:  tensor(-19.7229, device='cuda:1')
current relative error is:  tensor(0.0153, device='cuda:1')
current epoch is:  350
current loss is:  tensor(-20.9317, 

current epoch is:  3050
current loss is:  tensor(-20.9713, device='cuda:1')
current relative error is:  tensor(0.0332, device='cuda:1')
current epoch is:  3100
current loss is:  tensor(-20.7756, device='cuda:1')
current relative error is:  tensor(0.0281, device='cuda:1')
current epoch is:  3150
current loss is:  tensor(-21.7022, device='cuda:1')
current relative error is:  tensor(0.0361, device='cuda:1')
current epoch is:  3200
current loss is:  tensor(-19.1122, device='cuda:1')
current relative error is:  tensor(0.0702, device='cuda:1')
current epoch is:  3250
current loss is:  tensor(-18.9660, device='cuda:1')
current relative error is:  tensor(0.0246, device='cuda:1')
current epoch is:  3300
current loss is:  tensor(-19.7450, device='cuda:1')
current relative error is:  tensor(0.0250, device='cuda:1')
current epoch is:  3350
current loss is:  tensor(-20.0594, device='cuda:1')
current relative error is:  tensor(0.0255, device='cuda:1')
current epoch is:  3400
current loss is:  tensor

current epoch is:  6100
current loss is:  tensor(-20.0610, device='cuda:1')
current relative error is:  tensor(0.0264, device='cuda:1')
current epoch is:  6150
current loss is:  tensor(-18.8561, device='cuda:1')
current relative error is:  tensor(0.0179, device='cuda:1')
current epoch is:  6200
current loss is:  tensor(-17.9493, device='cuda:1')
current relative error is:  tensor(0.0223, device='cuda:1')
current epoch is:  6250
current loss is:  tensor(-19.0222, device='cuda:1')
current relative error is:  tensor(0.0606, device='cuda:1')
current epoch is:  6300
current loss is:  tensor(-19.1587, device='cuda:1')
current relative error is:  tensor(0.0292, device='cuda:1')
current epoch is:  6350
current loss is:  tensor(-19.4869, device='cuda:1')
current relative error is:  tensor(0.0235, device='cuda:1')
current epoch is:  6400
current loss is:  tensor(-19.8973, device='cuda:1')
current relative error is:  tensor(0.0186, device='cuda:1')
current epoch is:  6450
current loss is:  tensor

current epoch is:  9150
current loss is:  tensor(-19.8496, device='cuda:1')
current relative error is:  tensor(0.0189, device='cuda:1')
current epoch is:  9200
current loss is:  tensor(-19.9632, device='cuda:1')
current relative error is:  tensor(0.0379, device='cuda:1')
current epoch is:  9250
current loss is:  tensor(-21.1242, device='cuda:1')
current relative error is:  tensor(0.0330, device='cuda:1')
current epoch is:  9300
current loss is:  tensor(-18.9562, device='cuda:1')
current relative error is:  tensor(0.0187, device='cuda:1')
current epoch is:  9350
current loss is:  tensor(-19.5398, device='cuda:1')
current relative error is:  tensor(0.0281, device='cuda:1')
current epoch is:  9400
current loss is:  tensor(-19.1083, device='cuda:1')
current relative error is:  tensor(0.0242, device='cuda:1')
current epoch is:  9450
current loss is:  tensor(-19.7674, device='cuda:1')
current relative error is:  tensor(0.0282, device='cuda:1')
current epoch is:  9500
current loss is:  tensor

current epoch is:  12150
current loss is:  tensor(-20.1302, device='cuda:1')
current relative error is:  tensor(0.0185, device='cuda:1')
current epoch is:  12200
current loss is:  tensor(-21.2206, device='cuda:1')
current relative error is:  tensor(0.0171, device='cuda:1')
current epoch is:  12250
current loss is:  tensor(-18.1226, device='cuda:1')
current relative error is:  tensor(0.0319, device='cuda:1')
current epoch is:  12300
current loss is:  tensor(-18.6604, device='cuda:1')
current relative error is:  tensor(0.0188, device='cuda:1')
current epoch is:  12350
current loss is:  tensor(-21.4386, device='cuda:1')
current relative error is:  tensor(0.0422, device='cuda:1')
current epoch is:  12400
current loss is:  tensor(-20.7837, device='cuda:1')
current relative error is:  tensor(0.0586, device='cuda:1')
current epoch is:  12450
current loss is:  tensor(-20.0989, device='cuda:1')
current relative error is:  tensor(0.0364, device='cuda:1')
current epoch is:  12500
current loss is:

current epoch is:  15150
current loss is:  tensor(-19.9122, device='cuda:1')
current relative error is:  tensor(0.0266, device='cuda:1')
current epoch is:  15200
current loss is:  tensor(-19.9631, device='cuda:1')
current relative error is:  tensor(0.0373, device='cuda:1')
current epoch is:  15250
current loss is:  tensor(-19.9452, device='cuda:1')
current relative error is:  tensor(0.0288, device='cuda:1')
current epoch is:  15300
current loss is:  tensor(-20.0947, device='cuda:1')
current relative error is:  tensor(0.0309, device='cuda:1')
current epoch is:  15350
current loss is:  tensor(-20.0573, device='cuda:1')
current relative error is:  tensor(0.0640, device='cuda:1')
current epoch is:  15400
current loss is:  tensor(-18.5856, device='cuda:1')
current relative error is:  tensor(0.0182, device='cuda:1')
current epoch is:  15450
current loss is:  tensor(-20.4838, device='cuda:1')
current relative error is:  tensor(0.0255, device='cuda:1')
current epoch is:  15500
current loss is:

current epoch is:  18150
current loss is:  tensor(-19.9953, device='cuda:1')
current relative error is:  tensor(0.0132, device='cuda:1')
current epoch is:  18200
current loss is:  tensor(-18.6169, device='cuda:1')
current relative error is:  tensor(0.0354, device='cuda:1')
current epoch is:  18250
current loss is:  tensor(-19.6459, device='cuda:1')
current relative error is:  tensor(0.0126, device='cuda:1')
current epoch is:  18300
current loss is:  tensor(-21.0616, device='cuda:1')
current relative error is:  tensor(0.0244, device='cuda:1')
current epoch is:  18350
current loss is:  tensor(-19.8158, device='cuda:1')
current relative error is:  tensor(0.0117, device='cuda:1')
current epoch is:  18400
current loss is:  tensor(-18.5689, device='cuda:1')
current relative error is:  tensor(0.0295, device='cuda:1')
current epoch is:  18450
current loss is:  tensor(-20.9896, device='cuda:1')
current relative error is:  tensor(0.0159, device='cuda:1')
current epoch is:  18500
current loss is: